# Create Biomarker Supervisor Agent
In this notebook we create the Biomarker Supervisor Agent that will interact with all of the Biomarker sub-agents using the Strands agents. The agents will be deployed using Bedrock AgentCore.

#### Upgrade boto3 to the latest version with support for Bedrock AgentCore

In [ ]:
%pip install --upgrade boto3

#### Ensure the latest version of boto3 is shown below
Ensure the boto3 version printed below is **1.39** or higher.

In [ ]:
%pip show boto3

#### Install Strands agents and AgentCore dependencies

In [ ]:
%pip install strands-agents strands-agents-tools bedrock-agentcore bedrock-agentcore-starter-toolkit xmltodict --quiet

## Prerequisites

Run through the notebook environment setup in [00-setup_environment.ipynb](00-setup_environment.ipynb). You will also need to create an IAM role named **`agentcore-agentcore_strands-role`** for AgentCore runtime with the following policies:
- managed policy `AmazonEC2ContainerRegistryFullAccess`
- managed policy `AmazonRedshiftQueryEditor`
- `bedrock-agentcore:*`
- `bedrock:InvokeModel`
- `bedrock:InvokeModelWithResponseStream`
- `bedrock:ListKnowledgeBases`
- `lambda:InvokeFunction`
- `s3:GetObject`

#### Import required libraries

In [ ]:
from utils.magic_helper import register_cell_magic

## Agent Creation
In this section we create the supervisor agent

### Agents as Tools with Strands Agents

"Agents as Tools" is an architectural pattern in AI systems where specialized AI agents are wrapped as callable functions (tools) that can be used by other agents. This creates a hierarchical structure where:

1. A primary "orchestrator" agent handles user interaction and determines which specialized agent to call
2. Specialized "tool agents" perform domain-specific tasks when called by the orchestrator

This approach mimics human team dynamics, where a manager coordinates specialists, each bringing unique expertise to solve complex problems. Rather than a single agent trying to handle everything, tasks are delegated to the most appropriate specialized agent.

In [ ]:
%%write_and_run multi_agent_biomarker.py

import boto3
import json
import uuid
import requests
from typing import Dict, Any
from strands import Agent, tool
from strands.models import BedrockModel

from biomarker_agent import *
from clinical_research_agent import *
from medical_imaging_agent import *
from statistician_agent import *

# 1: biomarker_database_analyst_agent tool

@tool
def biomarker_database_analyst_agent(query: str) -> str:
    """
    Create biomarker query engine with redshift using Strands framework

    Args:
        query: An information request from the biomarker database

    Returns:
        A summary of the understanding of the user's query and the response.
    """
    try:
        biomarker_agent = Agent(
            model=bedrock_model,
            tools=[get_schema, query_redshift, refine_sql],
            system_prompt=biomarker_agent_instruction
        )
        biomarker_agent_response = biomarker_agent(query)
        print("Biomarker agent Response:")
        print(biomarker_agent_response)
        return biomarker_agent_response
    except Exception as e:
        print(f"Error creating agent: {e}")
        raise

# 2: clinical_evidence_research_agent tool

@tool
def clinical_evidence_research_agent(query: str) -> str:
    """
    Research internal and external evidence using Strands framework

    Args:
        query: An information request from the clinical evidence

    Returns:
        Clinical evidence.
    """
    try:
        clinical_research_agent = Agent(
            model=bedrock_model,
            tools=[query_pubmed, retrieve],
            system_prompt=clinical_research_agent_instruction
        )
        clinical_research_agent_response = clinical_research_agent(query)
        print("Clinical research agent Response:")
        print(clinical_research_agent_response)
        return clinical_research_agent_response
    except Exception as e:
        print(f"Error creating agent: {e}")
        raise

# 3: clinical_evidence_research_agent tool

@tool
def medical_imaging_agent(query: str) -> str:
    """
    Medical research assistant AI specialized in processing medical imaging scans of patients

    Args:
        query: Patient information to be used to compute imaging biomarkers

    Returns:
        The results of the medical imaging jobs.
    """
    try:
        medical_imaging_agent = Agent(
            model=bedrock_model,
            tools=[compute_imaging_biomarker, analyze_imaging_biomarker],
            system_prompt=medical_imaging_agent_instruction
        )
        medical_imaging_agent_response = medical_imaging_agent(query)
        print("Medical imaging agent Response:")
        print(medical_imaging_agent_response)
        return medical_imaging_agent_response
    except Exception as e:
        print(f"Error creating agent: {e}")
        raise

# 4: statistician_agent tool

@tool
def statistician_agent(query: str) -> str:
    """
    Medical research assistant AI specialized in survival analysis with biomarkers

    Args:
        query: Information to be used to generate the chart

    Returns:
        A summary of your understanding of the user's query.
    """
    try:
        statistician_agent = Agent(
            model=bedrock_model,
            tools=[create_bar_chart, plot_kaplan_meier, fit_survival_regression],
            system_prompt=statistician_agent_instruction
        )
        statistician_agent_response = statistician_agent(query)
        print("Statistician agent Response:")
        print(statistician_agent_response)
        return statistician_agent_response
    except Exception as e:
        print(f"Error creating agent: {e}")
        raise

# Define orchestrator agent configuration below

agent_name = "multi-agent-biomarker"
agent_description = "Multi-agent collaboration for biomarker discovery"
agent_instruction = """You are a medical research assistant AI specialized in cancer biomarker analysis and discovery. 
Your primary task is to interpret user queries, use relevant agents for specific tasks, and provide consolidated medical insights based on the data. 
Use only the appropriate agents as required by the specific question. You can provide responses from a prior agent to the next agent 
in sequence. To analyze patient biomarkers data, you can retrieve relevant records from the database. 
To find the p-value of biomarkers, 
a. You need to query and store all records including survival status, survival duration in years, and the required biomarkers and 
b. You need to fit a surival regression model with that data in S3. 
When providing your response:
a. Start with a brief summary of your understanding of the user's query. 
b. Explain the steps you're taking to address the query. Ask for clarifications from the user if required. 
c. Present the results of individual agents 
d. Conclude with a concise summary of the findings and their potential implications for medical research. 

Make sure to explain any medical or statistical concepts in a clear, accessible manner.

"""

# Define the model
bedrock_model = BedrockModel(
    model_id="anthropic.claude-3-5-sonnet-20241022-v2:0",
    region_name=region,
    temperature=0.1,
    streaming=False
)

# Instantiate the orchestrator agent
try:
    orchestrator = Agent(
        model=bedrock_model,
        system_prompt=agent_instruction,
        callback_handler=None,
        # associate sub-agents to supervisor
        tools=[biomarker_database_analyst_agent, clinical_evidence_research_agent, medical_imaging_agent, statistician_agent]
    )
    print(f"Successfully created orchestrator agent: {agent_name}")
except Exception as e:
    print(f"Error creating agent: {e}")
    raise

### Ask supervisor agent different questions now that sub-agents are ready

In [ ]:
# ---------------------------- Sample Question Bank --------------------------------------------

# Redshift Agent Questions
redshift_agent_query_1 = "How many patients are current smokers?"
redshift_agent_query_2 = "What is the average age of patients diagnosed with Adenocarcinoma?"

# Research Evidence Agent Questions
research_evidence_agent_query_1 = "Can you search PubMed for evidence around the effects of biomarker use in oncology on clinical trial failure risk?"
research_evidence_agent_query_2 = "What are the FDA approved biomarkers for non small cell lung cancer?"

# Medical Imaging Agent Questions (must run in sequence)
medical_imaging_agent_query_1 = "Can you compute the imaging biomarkers for the 2 patients with the lowest gdf15 expression values?"
medical_imaging_agent_query_2 = "Can you higlight the elongation and sphericity of the tumor with these patients. Can you depict images of them?"

# Scientific Analysis Agent Questions
scientific_analysis_agent_query_1 = "What is the best gene biomarker (lowest p value) with overall survival for patients that have undergone chemotherapy, Generate a bar chart of the top 5 gene biomarkers based on their p value and include their names in the x axis.?"

# Followup research evidence agent questions
research_evidence_agent_query_3 = "According to literature evidence, what metagene cluster does gdf15 belong to"
research_evidence_agent_query_4 = "What properties of the tumor are associated with metagene 19 activity and EGFR pathway"

# -----------------------------------------------------------------------------------------
test_query = redshift_agent_query_1 # Change value here to test different questions

print(f"Testing orchestrator agent with query: {test_query}")
print("=" * (39 + len(test_query)))

try:
    # Run the agent
    response = orchestrator(test_query)
    
except Exception as e:
    print(f"Error during agent execution: {e}")
    import traceback
    traceback.print_exc()

## Agent Deployment
In this section we deploy the supervisor agent using Bedrock AgentCore.

### Preparing your agent for deployment on AgentCore Runtime

In [ ]:
%%writefile -a multi_agent_biomarker.py

import logging
import strands
import argparse
import json
import uuid
from bedrock_agentcore.runtime import BedrockAgentCoreApp

# Configure the root strands logger
logging.getLogger("strands").setLevel(logging.DEBUG)

# Add a handler to see the logs
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()]
)

# Custom event serializer function
def json_serializer(obj):
    if isinstance(obj, strands.agent.agent.Agent):
        return obj.name
    elif isinstance(obj, strands.agent.agent_result.AgentResult):
        result = {}
        result["message"] = str(obj)
        result["metrics"] = {}
        result["metrics"]["accumulated_usage"] = obj.metrics.accumulated_usage
        result["metrics"]["accumulated_metrics"] = obj.metrics.accumulated_metrics
        return result
    elif isinstance(obj, uuid.UUID):
        return str(obj)
    else:
        # Assume other objects are not serializable
        return None

# Get a logger instance
logger = logging.getLogger(__name__)

app = BedrockAgentCoreApp()

@app.entrypoint
async def strands_agent_bedrock_streaming(payload):
    """
    Invoke the agent with streaming capabilities
    This function demonstrates how to implement streaming responses
    with AgentCore Runtime using async generators
    """
    user_input = payload.get("prompt")
    logger.info("User input:", user_input)
    
    try:
        # Stream each chunk as it becomes available
        async for event in orchestrator.stream_async(user_input):
            logger.debug("Received event:", event)
            # serialize the event object
            yield json.dumps(event, default=json_serializer)
    except Exception as e:
        # Handle errors gracefully in streaming context
        error_response = {"error": str(e), "type": "stream_error"}
        logger.error(f"Streaming error: {error_response}")
        yield error_response

if __name__ == "__main__":
    app.run()


### Deploying the agent to AgentCore Runtime

#### Define agent name and retrieve runtime role

In [ ]:
from utils.boto3_helper import get_role_arn
iam = boto3.client('iam')

agent_name="agentcore_strands"
agentcore_iam_role = get_role_arn('BedrockAgentCoreStrands')
agentcore_iam_role

#### Configure AgentCore Runtime deployment
During the configure step, your docker file will be generated based on your application code.

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    entrypoint="multi_agent_biomarker.py",
    execution_role=agentcore_iam_role,
    auto_create_ecr=True,
    requirements_file="runtime_requirements.txt",
    region=region,
    agent_name=agent_name
)
response

#### Launching agent to AgentCore Runtime
Now that we've got a docker file, let's launch the agent to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime.

In [ ]:
launch_result = agentcore_runtime.launch(
    auto_update_on_conflict=True
)
launch_result

Let's also register the agent ARN in SSM Parameter Store so it can be used by other services.

In [ ]:
# Update AGENT_ID parameter in SSM
ssm_client = boto3.client('ssm', region)
ssm_client.put_parameter(
    Name='/streamlitapp/env1/AGENT_ARN',
    Value=launch_result.agent_arn,
    Type='String',
    Overwrite=True
)

### Now the Biomarker Supervisor Agent is ready to assist you!

#### Invoking AgentCore Runtime
Finally, we can invoke our AgentCore Runtime with a payload.

In [ ]:
invoke_response = agentcore_runtime.invoke({"prompt": redshift_agent_query_1})


### Invoking AgentCore Runtime with boto3
Now that your AgentCore Runtime was created you can invoke it with any AWS SDK. For instance, you can use the boto3 `invoke_agent_runtime` method for it.

In [ ]:
import json
import boto3
from IPython.display import Markdown, display

agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": test_query})
)

print(f"Response contentType: {response.get("contentType")}\n")
print(f"Testing orchestrator agent boto3 client: {test_query}")
print("=" * (41 + len(test_query)))

if "text/event-stream" in response.get("contentType", ""):
    # Processing streaming response
    for line in response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                # remove the SSE structure
                data = line[6:]
                # we need to parse it twice to convert from JSON str to a dictionary
                data_obj = json.loads(data)
                data_obj = json.loads(data_obj)
                # for this example we only care about the data field
                if "data" in data_obj:
                    print(data_obj.get("data"))
else:
    # Handle non-streaming response
    try:
        events = []
        for event in response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]
    if events:
        try:
            response_data = json.loads(events[0].decode("utf-8"))
            display(Markdown(response_data))
        except:
            print(f"Raw response: {events[0]}")